In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.append("/content/drive/MyDrive/CS224N_Reverse_Dictionary-main/src")

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.3 MB/s eta 0:00:00


In [4]:
from transformers import BertTokenizer, BertForMaskedLM
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.nn import functional as F
from tqdm import tqdm
import torch
import numpy as np
import time
import pandas as pd

from torch.utils.data import random_split, DataLoader, RandomSampler, SequentialSampler
from data import CustomDataset, dataset_tokenized, get_wordvecs
from models.model_2 import Model2

In [6]:
path = "/content/drive/MyDrive/CS224N_Reverse_Dictionary-main/toyset10000.csv"
df = pd.read_csv(path, dtype="string")
df['Definition'] = df['Definition'].astype(str)
df = df[['Word', 'Definition']]
df.shape

(34828, 2)

In [7]:
df[~df['Word'].str.contains('_')]
df['Word'] = df['Word'].apply(lambda x: x.lower())
df['Definition'] = df['Definition'].apply(str.lower)
df[~df['Word'].str.contains('\"')]
df = df.applymap(lambda x: x.replace('\"', ''))
df['Definition'] = df['Definition'].apply(lambda x : x.split(';'))
df = df.explode('Definition')
df.reset_index()
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df

,Word,Definition
0,lapidary,an artificer who cuts polishes and engraves p...
1,ease,satisfaction pleasure hence accommodation ente...
2,occult,hidden from view concealed
3,phaeochromocytoma,a vascular tumor of the adrenal gland
4,distancing,the length of a race especially of a horserace
...,...,...
41479,amide,a compound with a metal replacing hydrogen in ...
41480,formalize,declare or make legally valid
41481,beaux_arts,the study and creation of visual works of art
41482,toxicity,the degree to which something is poisonous


In [8]:
if torch.cuda.is_available():    

    device = torch.device("cuda")

else:
    device = torch.device("cpu")

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [10]:
import gensim.downloader as api

wv_from_bin = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [11]:
contains_glove = []
for _, word in df['Word'].items():
  if word in wv_from_bin:
    contains_glove += [True]
  else:
    contains_glove += [False]
df = df[contains_glove]
df.shape

(32872, 2)

In [12]:
input_ids = []
attention_masks = []
max_len = 200
for sent in df['Definition']:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, 
                        max_length = 200,          
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt'
                   )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [13]:
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

In [14]:
torch.save(input_ids, 'input_ids_cleaned')
torch.save(attention_masks, 'attention_masks_cleaned')

In [15]:
import numpy as np
labels = []
count = 0
for word in df['Word']:
  try:
    label = wv_from_bin[word]
  except:
    count += 1
    label = [np.random.rand() * 2 - 1 for i in range(100)]
  labels.append(label)
labels = np.array(labels)
labels = torch.tensor(labels)
print(count)

0


In [16]:
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, labels)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size], generator=torch.Generator().manual_seed(224))

test_size = int(0.5 * val_size)
val_size = val_size - test_size

val_dataset, test_dataset = random_split(val_dataset, [val_size, test_size], generator=torch.Generator().manual_seed(224))

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
print('{:>5,} test samples'.format(test_size))

26,297 training samples
3,288 validation samples
3,287 test samples


In [17]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16

train_dataloader = DataLoader(
            train_dataset, 
            sampler = RandomSampler(train_dataset, generator=torch.Generator().manual_seed(224)),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), 
            batch_size = batch_size 
        )

test_dataloader = DataLoader(
            test_dataset, 
            sampler = SequentialSampler(test_dataset), 
            batch_size = batch_size
        )

In [21]:
model = Model2()
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model2(
  (bert_backend): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [22]:
from transformers import AdamW, BertConfig
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8
                )

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [23]:
from transformers import get_linear_schedule_with_warmup

epochs = 15
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [24]:
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [25]:
loss_fn = torch.nn.MSELoss(reduction='sum')

In [ ]:
import random
import numpy as np

seed_val = 224

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []
total_t0 = time.time()
for epoch_i in range(0, epochs):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        if step % 50 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            torch.save(model.state_dict(), "/content/drive/MyDrive/CS224N_Reverse_Dictionary-main/model.params.f")
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()        

        output = model(b_input_ids, b_input_mask)
        batch_loss = loss_fn(output, b_labels.float())
        total_train_loss += batch_loss.item()
        batch_loss.double().backward()

        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader.dataset)            
    
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

    top_count = 0
    top_10_count = 0
    top_100_count = 0
        
    print("")
    print("Running Validation...")

    t0 = time.time()
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        with torch.no_grad(): 
            outputs = model(b_input_ids, b_input_mask)
            total_eval_loss += loss_fn(outputs, b_labels.float())
        for i in range(outputs.size(0)): 
            top_100 = wv_from_bin.most_similar(positive=[outputs[i].cpu().numpy().astype(np.float32)], topn=100)
            top_10 = wv_from_bin.most_similar(positive=[outputs[i].cpu().numpy().astype(np.float32)], topn=10)[:10]
            # top_10 = top_100[:10]
            actual = wv_from_bin.most_similar(positive=[b_labels[i].cpu().numpy().astype(np.float32)], topn=1)[0][0]
            
            top_count += int(actual == top_10[0][0])
            top_10_count += int(actual in [pairs[0] for pairs in top_10])
            top_100_count += int(actual in [pairs[0] for pairs in top_100])

    avg_val_accuracy = top_count / len(validation_dataloader.dataset)
    print("  Top 1 Accuracy: {0:.2f}".format(avg_val_accuracy))
    avg_val_10_accuracy = top_10_count / len(validation_dataloader.dataset)
    print("  Top 10 Accuracy: {0:.2f}".format(avg_val_10_accuracy))
    avg_val_100_accuracy = top_100_count / len(validation_dataloader.dataset)
    print("  Top 100 Accuracy: {0:.2f}".format(avg_val_100_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader.dataset)

    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Top 1 Accur.': avg_val_accuracy,
            'Valid. Top 10 Accur.': avg_val_10_accuracy,
            'Valid. Top 100 Accur.': avg_val_100_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 15 ========
Training...
  Batch    50  of  1,644.    Elapsed: 0:00:26.
  Batch   100  of  1,644.    Elapsed: 0:01:03.
  Batch   150  of  1,644.    Elapsed: 0:01:32.
  Batch   200  of  1,644.    Elapsed: 0:02:03.
  Batch   250  of  1,644.    Elapsed: 0:02:33.
  Batch   300  of  1,644.    Elapsed: 0:03:04.
  Batch   350  of  1,644.    Elapsed: 0:03:34.
  Batch   400  of  1,644.    Elapsed: 0:04:04.
  Batch   450  of  1,644.    Elapsed: 0:04:35.
  Batch   500  of  1,644.    Elapsed: 0:05:05.
  Batch   550  of  1,644.    Elapsed: 0:05:35.
  Batch   600  of  1,644.    Elapsed: 0:06:05.
  Batch   650  of  1,644.    Elapsed: 0:06:35.
  Batch   700  of  1,644.    Elapsed: 0:07:05.
  Batch   750  of  1,644.    Elapsed: 0:07:35.
  Batch   800  of  1,644.    Elapsed: 0:08:05.
  Batch   850  of  1,644.    Elapsed: 0:08:34.
  Batch   900  of  1,644.    Elapsed: 0:09:04.
  Batch   950  of  1,644.    Elapsed: 0:09:34.
  Batch 1,000  of  1,644.    Elapsed: 0:10:04.
  Batch 1,050  o

In [18]:
model = Model2()
model.load_state_dict(torch.load("/content/drive/MyDrive/CS224N_Reverse_Dictionary-main/model.params.f",map_location=torch.device('cpu')))
# model.load_state_dict(torch.load("/content/drive/MyDrive/CS224N_Reverse_Dictionary-main/model.params.f"))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [32]:
print("Running Test Loader...")
top_count = 0
top_10_count = 0
top_100_count = 0
t0 = time.time()

model.eval()

total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

for batch in test_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    with torch.no_grad():        
        outputs = model(b_input_ids, b_input_mask)
    
    for i in range(outputs.size(0)): # batch_size
      top_100 = wv_from_bin.most_similar(positive=[outputs[i].cpu().numpy().astype(np.float32)], topn=100)
      top_10 = wv_from_bin.most_similar(positive=[outputs[i].cpu().numpy().astype(np.float32)], topn=10)[:10]
      actual = wv_from_bin.most_similar(positive=[b_labels[i].cpu().numpy().astype(np.float32)], topn=1)[0][0]
      top_count += int(actual == top_10[0][0])
      top_10_count += int(actual in [pairs[0] for pairs in top_10])
      top_100_count += int(actual in [pairs[0] for pairs in top_100])
    

# Report the final accuracy for this validation run.
avg_val_accuracy = top_count / len(test_dataloader.dataset)
print("  Top 1 Accuracy: {0:.2f}".format(avg_val_accuracy))
avg_val_10_accuracy = top_10_count / len(test_dataloader.dataset)
print("  Top 10 Accuracy: {0:.2f}".format(avg_val_10_accuracy))
avg_val_100_accuracy = top_100_count / len(test_dataloader.dataset)
print("  Top 100 Accuracy: {0:.2f}".format(avg_val_100_accuracy))

validation_time = format_time(time.time() - t0)

print("  Test took: {:}".format(validation_time))

Running Test Loader...
  Top 1 Accuracy: 0.22
  Top 10 Accuracy: 0.39
  Top 100 Accuracy: 0.57
  Test took: 0:41:26


In [19]:
# sentence = "causing laughter or amusement"
# sentence = "sail or travel over "
# sentence = "travel along a particular direction"
sentence = "structure forming the upper covering of a builting"

batch_size = 16
test_input_ids = []
test_attention_masks = []

test_encoded_dict = tokenizer.encode_plus(
                    sentence,                     
                    add_special_tokens = True,
                    max_length = 200, 
                    pad_to_max_length = True,
                    return_attention_mask = True,  
                    return_tensors = 'pt',
                ) 
test_input_ids.append(test_encoded_dict['input_ids'])
test_attention_masks.append(test_encoded_dict['attention_mask'])
test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim = 0)
test = TensorDataset(test_input_ids, test_attention_masks)
test = DataLoader(
            test, # The test samples.
            sampler = SequentialSampler(test),
            batch_size = batch_size
        )

In [20]:
for batch in test:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    with torch.no_grad():        
      outputs = model(b_input_ids, b_input_mask)
      op = wv_from_bin.most_similar(positive=[outputs[0].cpu().numpy().astype(np.float32)], topn=10)
      ops = []
      for word, prob in op:
        ops.append(word)
print(ops)

['roof', 'walls', 'underneath', 'roofs', 'brick', 'slab', 'covered', 'beneath', 'wooden', 'shaped']


In [21]:
path2 = "/content/drive/MyDrive/CS224N_Reverse_Dictionary-main/testset.csv"
testdf = pd.read_csv(path2, dtype="string")
testdf

,Word,Definition
0,individualized,made to fit someone's needs
1,aspire,dream to achieve
2,being,to be existing
3,wipe,clean by rubbing
4,trembling,shaking with fear
...,...,...
95,dead,opposite of alive
96,dismantle,opposite of construct
97,abrupt,opposite of gradual
98,rebuke,opposite of endorse


In [22]:
test_input_ids = []
test_attention_masks = []
test_labels = []
i=0
for sent in testdf['Definition']:
    
    test_encoded_dict = tokenizer.encode_plus(
                        sent,                     
                        add_special_tokens = True, 
                        max_length = 200, 
                        pad_to_max_length = True,
                        return_attention_mask = True, 
                        return_tensors = 'pt', 
                    ) 
    test_input_ids.append(test_encoded_dict['input_ids'])
    test_attention_masks.append(test_encoded_dict['attention_mask'])
    word = testdf['Word'][i]
    test_labels.append(wv_from_bin[word])
    i+=1

test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim = 0)

test_labels = torch.tensor(test_labels)
test = TensorDataset(test_input_ids, test_attention_masks, test_labels)
test = DataLoader(
            test,
            sampler = SequentialSampler(test), 
            batch_size = batch_size
        )
    
j=0
top_count = 0
top_10_count = 0
top_100_count = 0
for batch in test:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    with torch.no_grad():        
      outputs = model(b_input_ids, b_input_mask)
      
    for i in range(outputs.size(0)): 
          top_100 = wv_from_bin.most_similar(positive=[outputs[i].cpu().numpy().astype(np.float32)], topn=100)
          top_10 = wv_from_bin.most_similar(positive=[outputs[i].cpu().numpy().astype(np.float32)], topn=10)[:10]
          actual = testdf['Word'][j]
          
          top_count += int(actual == top_10[0][0])
          top_10_count += int(actual in [pairs[0] for pairs in top_10])
          top_100_count += int(actual in [pairs[0] for pairs in top_100])
          j += 1

avg_val_accuracy = top_count / len(test.dataset)
print("  Top 1 Accuracy: {0:.2f}".format(avg_val_accuracy))
avg_val_10_accuracy = top_10_count / len(test.dataset)
print("  Top 10 Accuracy: {0:.2f}".format(avg_val_10_accuracy))
avg_val_100_accuracy = top_100_count / len(test.dataset)
print("  Top 100 Accuracy: {0:.2f}".format(avg_val_100_accuracy))
  

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
<ipython-input-22-c97982fea79d>:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  test_labels = torch.tensor(test_labels)


  Top 1 Accuracy: 0.04
  Top 10 Accuracy: 0.06
  Top 100 Accuracy: 0.20
